In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from detectron2.data import DatasetCatalog, MetadataCatalog

def infuse_depth_into_rgb(image_array, depth_array):
    """
    Infuses depth information into the blue channel of RGB images.

    :param image_array: List of RGB images (as numpy arrays)
    :param depth_array: List of depth maps corresponding to the images
    :return: List of RGB images with depth infused in the blue channel
    """
    image_array_infused = []

    for i in tqdm(range(len(image_array)), desc="Infusing Images"):
        image = image_array[i]
        depth_map = depth_array[i]

        # Resize the depth map to match the image dimensions
        depth_map_resized = cv2.resize(depth_map, (image.shape[1], image.shape[0]))

        # Ensure depth map is single channel (grayscale)
        if len(depth_map_resized.shape) == 3:
            depth_map_resized = cv2.cvtColor(depth_map_resized, cv2.COLOR_BGR2GRAY)

        # Split the image into RGB channels
        b, g, r = cv2.split(image)

        # Normalize depth map to match the blue channel (0-255) and convert to uint8
        depth_map_normalized = cv2.normalize(depth_map_resized, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        # Infuse the depth map into the blue channel
        if b.shape == depth_map_normalized.shape:
            infused_blue = cv2.addWeighted(b, 0.5, depth_map_normalized, 0.5, 0)
        else:
            raise ValueError("Dimension mismatch between the blue channel and the depth map.")

        # Merge the channels back
        infused_image = cv2.merge((infused_blue, g, r))

        image_array_infused.append(infused_image)

    return image_array_infused

def load_images_and_depth(image_dir, depth_dir):
    """
    Loads images and corresponding depth maps from directories.
    
    :param image_dir: Directory containing RGB images.
    :param depth_dir: Directory containing depth maps.
    :return: List of RGB images and corresponding depth maps.
    """
    image_array = []
    depth_array = []

    # Load images and depth maps
    image_filenames = sorted(os.listdir(image_dir))
    depth_filenames = sorted(os.listdir(depth_dir))

    for img_file, depth_file in zip(image_filenames, depth_filenames):
        img_path = os.path.join(image_dir, img_file)
        depth_path = os.path.join(depth_dir, depth_file)

        # Read the RGB image and depth map
        image = cv2.imread(img_path)
        depth_map = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)

        image_array.append(image)
        depth_array.append(depth_map)

    return image_array, depth_array

def register_rgbd_dataset(image_dir, depth_dir, annotation_json, dataset_name, split="train"):
    """
    Registers a Detectron2 dataset where RGB images are infused with depth information.

    :param image_dir: Directory with RGB images.
    :param depth_dir: Directory with depth maps.
    :param annotation_json: Path to COCO-style annotations JSON file.
    :param dataset_name: Name of the dataset for Detectron2.
    :param split: Train, validation, or test split.
    """
    # Load the images and depth maps
    image_array, depth_array = load_images_and_depth(image_dir, depth_dir)

    # Infuse depth maps into the blue channel
    image_array_infused = infuse_depth_into_rgb(image_array, depth_array)

    # Save infused images into a directory
    infused_image_dir = os.path.join(image_dir, f"infused_{split}")
    if not os.path.exists(infused_image_dir):
        os.makedirs(infused_image_dir)

    for i, infused_image in enumerate(image_array_infused):
        infused_image_path = os.path.join(infused_image_dir, f"infused_{i}.png")
        cv2.imwrite(infused_image_path, infused_image)

    # Register the dataset with Detectron2
    register_coco_instances(dataset_name, {}, annotation_json, infused_image_dir)

# Example usage:
# Specify your directories
image_dir = "../data/rgb_images"
depth_dir = "../data/depth_maps"
annotation_json = "../data/coco_annotations/annotations.json"

# Register the dataset for training
register_rgbd_dataset(image_dir, depth_dir, annotation_json, "my_rgbd_dataset_train", split="train")

# Register the dataset for validation
register_rgbd_dataset(image_dir, depth_dir, annotation_json, "my_rgbd_dataset_val", split="val")
